In [1]:
# earth engine things
import ee
import geemap
ee.Initialize()

In [3]:
# create interactive map
Map = geemap.Map(center=[-13.1, 27.8], zoom=6)

# load Hansen dataset
dataset = ee.Image('UMD/hansen/global_forest_change_2023_v1_11')

# load Zambia boundary
zambia = ee.FeatureCollection('FAO/GAUL/2015/level0') \
            .filter(ee.Filter.eq('ADM0_NAME', 'Zambia'))

# clip dataset to Zambia
clipped_dataset = dataset.clip(zambia)

In [7]:
# visualization parameters
tree_cover_vis = {
    'bands': ['treecover2000'],
    'min': 0,
    'max': 100,
    'palette': ['black', 'green']
}

loss_vis = {
    'min': 0,
    'max': 23,
    'palette': ['yellow', 'red']
}

In [10]:
# add base tree cover layer
Map.addLayer(clipped_dataset.select('treecover2000'), tree_cover_vis, 'Tree Cover 2000')

# loop through 'test years'
for year in range(2001, 2021, 5):
    code = year - 2000
    annual_loss = clipped_dataset.select('lossyear') \
                        .eq(code) \
                        .selfMask()
    Map.addLayer(annual_loss, {'palette': ['red']}, f'Tree Loss {year}')

Map

Map(center=[-13.1, 27.8], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…